In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split # Divide el dataset en testeo y entrenamiento

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRFRegressor

from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
#Descargar datos
!wget https://raw.githubusercontent.com/jsrpy/NYC-Property-Regression/master/real_estate_data.csv

--2022-10-06 11:21:38--  https://raw.githubusercontent.com/jsrpy/NYC-Property-Regression/master/real_estate_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252295 (246K) [text/plain]
Saving to: ‘real_estate_data.csv’

real_estate_data.cs 100%[===================>] 246.38K  --.-KB/s    in 0.03s   

2022-10-06 11:21:38 (7.84 MB/s) - ‘real_estate_data.csv’ saved [252295/252295]



In [ ]:
df = pd.read_csv('real_estate_data.csv')
df.head()

,tx_price,beds,baths,sqft,year_built,lot_size,property_type,exterior_walls,roof,basement,...,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,tx_year
0,295850,1,1,584,2013,0,Apartment / Condo / Townhouse,Wood Siding,NaN,NaN,...,47,58,33.0,65.0,84.0,234.0,81.0,9.0,3.0,2013
1,216500,1,1,612,1965,0,Apartment / Condo / Townhouse,Brick,Composition Shingle,1.0,...,26,14,39.0,73.0,69.0,169.0,51.0,3.0,3.0,2006
2,279900,1,1,615,1963,0,Apartment / Condo / Townhouse,Wood Siding,NaN,NaN,...,74,62,28.0,15.0,86.0,216.0,74.0,8.0,3.0,2012
3,379900,1,1,618,2000,33541,Apartment / Condo / Townhouse,Wood Siding,NaN,NaN,...,72,83,36.0,25.0,91.0,265.0,92.0,9.0,3.0,2005
4,340000,1,1,634,1992,0,Apartment / Condo / Townhouse,Brick,NaN,NaN,...,50,73,37.0,20.0,75.0,88.0,30.0,9.0,3.0,2002


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tx_price            1883 non-null   int64  
 1   beds                1883 non-null   int64  
 2   baths               1883 non-null   int64  
 3   sqft                1883 non-null   int64  
 4   year_built          1883 non-null   int64  
 5   lot_size            1883 non-null   int64  
 6   property_type       1883 non-null   object 
 7   exterior_walls      1660 non-null   object 
 8   roof                1529 non-null   object 
 9   basement            1657 non-null   float64
 10  restaurants         1883 non-null   int64  
 11  groceries           1883 non-null   int64  
 12  nightlife           1883 non-null   int64  
 13  cafes               1883 non-null   int64  
 14  shopping            1883 non-null   int64  
 15  arts_entertainment  1883 non-null   int64  
 16  beauty

#Manejo de datos nulos

In [ ]:
# Preguntar que filas tienen datos nulos
df.isnull().any()

tx_price              False
beds                  False
baths                 False
sqft                  False
year_built            False
lot_size              False
property_type         False
exterior_walls         True
roof                   True
basement               True
restaurants           False
groceries             False
nightlife             False
cafes                 False
shopping              False
arts_entertainment    False
beauty_spas           False
active_life           False
median_age            False
married               False
college_grad          False
property_tax          False
insurance             False
median_school         False
num_schools           False
tx_year               False
dtype: bool

In [ ]:
print(sum(df.exterior_walls.isnull())) #Es equivalente a df['exterior_walls']
print(sum(df.roof.isnull()))
print(sum(df.basement.isnull()))


223
354
226


In [ ]:
#Devuelve datos estadísticos de las columnas numéricas
df.describe()

,tx_price,beds,baths,sqft,year_built,lot_size,basement,restaurants,groceries,nightlife,...,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,tx_year
count,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1.883000e+03,1657.0,1883.000000,1883.000000,1883.000000,...,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000
mean,422839.807754,3.420605,2.579926,2329.398832,1982.963887,1.339262e+04,1.0,40.210303,4.505045,5.074881,...,23.416888,15.835369,38.601168,69.091875,65.085502,466.777483,140.454063,6.502921,2.793415,2007.111524
std,151462.593276,1.068554,0.945576,1336.991858,20.295945,4.494930e+04,0.0,46.867012,4.491029,8.464668,...,25.776916,17.667717,6.634110,19.659767,16.953165,231.656645,72.929765,1.996109,0.505358,5.196898
min,200000.000000,1.000000,1.000000,500.000000,1880.000000,0.000000e+00,1.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,22.000000,11.000000,5.000000,88.000000,30.000000,1.000000,1.000000,1993.000000
25%,300000.000000,3.000000,2.000000,1345.000000,1970.000000,1.542000e+03,1.0,7.000000,1.000000,0.000000,...,4.000000,4.000000,33.000000,58.000000,54.000000,320.000000,94.000000,5.000000,3.000000,2004.000000
50%,392000.000000,3.000000,3.000000,1907.000000,1986.000000,6.098000e+03,1.0,23.000000,3.000000,2.000000,...,15.000000,10.000000,38.000000,73.000000,66.000000,426.000000,125.000000,7.000000,3.000000,2007.000000
75%,525000.000000,4.000000,3.000000,3005.000000,2000.000000,1.176100e+04,1.0,58.000000,7.000000,6.000000,...,35.000000,21.000000,43.000000,84.000000,78.000000,569.000000,169.000000,8.000000,3.000000,2011.000000
max,800000.000000,5.000000,6.000000,8450.000000,2015.000000,1.220551e+06,1.0,266.000000,24.000000,54.000000,...,177.000000,94.000000,69.000000,100.000000,100.000000,4508.000000,1374.000000,10.000000,4.000000,2016.000000


In [ ]:
#Cuantas muestras tienen 0 en lot size
df.lot_size.isin([0]).sum()

352

In [ ]:
#Eliminamos columnas "basement" y "lot_size"
#Se borra lot_size porque el valor mínimo en la columna es 0 y hay 352 valores que son 0, como es un valor determinante para el precio de la casa, es
#mejor eliminarlo al tener tantos 0's, haciendo que el modelo se equivoque
df_filter = df.drop(['basement','lot_size'], axis=1)

##Tratamiento Columa Roof

In [ ]:
#Columna roof
df_filter.roof.unique()

array([nan, 'Composition Shingle', 'Other', 'Gravel/Rock',
       'Roll Composition', 'Asphalt', 'Slate', 'asphalt', 'Shake Shingle',
       'Built-up', 'Asbestos', 'Metal', 'shake-shingle',
       'Wood Shake/ Shingles', 'Composition', 'composition',
       'asphalt,shake-shingle'], dtype=object)

In [ ]:
df_filter.roof.value_counts()

Composition Shingle      1179
Asphalt                   132
Shake Shingle              55
Other                      49
Gravel/Rock                30
Wood Shake/ Shingles       30
Roll Composition           12
Slate                       9
Asbestos                    9
asphalt                     5
Composition                 5
Metal                       4
composition                 4
shake-shingle               3
Built-up                    2
asphalt,shake-shingle       1
Name: roof, dtype: int64

In [ ]:
# Reemplazamos los valores o categorías de una columna
df_filter.roof.replace('composition','Composition',inplace=True)
df_filter.roof.replace('asphalt','Asphalt',inplace=True)
df_filter.roof.replace(['asphalt,shake-shingle','shake-shingle', 'Wood Shake/ Shingles', 'Shake Shingle', 'Composition Shingle'],'Shingle',inplace=True)

In [ ]:
#Cuenta cuantos valores de cada categoría existen en la columna roof
df_filter.roof.value_counts()

Shingle             1268
Asphalt              137
Other                 49
Gravel/Rock           30
Roll Composition      12
Slate                  9
Asbestos               9
Composition            9
Metal                  4
Built-up               2
Name: roof, dtype: int64

In [ ]:
sum(df.roof.isnull())

354

In [ ]:
#Rellenar los NaN con 'newNaN'
df_filter.roof.fillna('newNAN', inplace=True)

In [ ]:
sum(df_filter.roof.isnull())

0

In [ ]:
df_filter.roof.value_counts()

Shingle             1268
newNAN               354
Asphalt              137
Other                 49
Gravel/Rock           30
Roll Composition      12
Slate                  9
Asbestos               9
Composition            9
Metal                  4
Built-up               2
Name: roof, dtype: int64

##Tratamiento Columna Exterior Walls

In [ ]:
#Columna Exterior walls
df_filter.exterior_walls.value_counts()

Brick                  687
Siding (Alum/Vinyl)    503
Metal                  120
Combination            107
Wood                    72
Wood Siding             49
Brick veneer            48
Stucco                  26
Other                   10
Concrete                 8
Concrete Block           7
Block                    7
Asbestos shingle         6
Rock, Stone              5
Masonry                  3
Wood Shingle             2
Name: exterior_walls, dtype: int64

In [ ]:
df_filter.exterior_walls.replace(['Wood Siding', 'Asbestos shingle', 'Wood Shingle'],'Wood',inplace=True)
df_filter.exterior_walls.replace(['Rock, Stone','Concrete', 'Concrete Block'],'Block',inplace=True)
df_filter.exterior_walls.replace(['Masonry', 'Brick veneer'],'Brick', inplace=True)

In [ ]:
df_filter.exterior_walls.value_counts()

Brick                  738
Siding (Alum/Vinyl)    503
Wood                   129
Metal                  120
Combination            107
Block                   27
Stucco                  26
Other                   10
Name: exterior_walls, dtype: int64

In [ ]:
sum(df_filter.exterior_walls.isnull())

223

In [ ]:
df_filter.exterior_walls.fillna('newNan',inplace=True)

In [ ]:
df_filter.exterior_walls.value_counts()

Brick                  738
Siding (Alum/Vinyl)    503
newNan                 223
Wood                   129
Metal                  120
Combination            107
Block                   27
Stucco                  26
Other                   10
Name: exterior_walls, dtype: int64

#Transformación y codificación de las columnas categóricas

In [ ]:
df_filter = pd.get_dummies(df_filter)
df_filter.head()

,tx_price,beds,baths,sqft,year_built,restaurants,groceries,nightlife,cafes,shopping,...,roof_Asphalt,roof_Built-up,roof_Composition,roof_Gravel/Rock,roof_Metal,roof_Other,roof_Roll Composition,roof_Shingle,roof_Slate,roof_newNAN
0,295850,1,1,584,2013,107,9,30,19,89,...,0,0,0,0,0,0,0,0,0,1
1,216500,1,1,612,1965,105,15,6,13,87,...,0,0,0,0,0,0,0,1,0,0
2,279900,1,1,615,1963,183,13,31,30,101,...,0,0,0,0,0,0,0,0,0,1
3,379900,1,1,618,2000,198,9,38,25,127,...,0,0,0,0,0,0,0,0,0,1
4,340000,1,1,634,1992,149,7,22,20,83,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 43 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   tx_price                                     1883 non-null   int64  
 1   beds                                         1883 non-null   int64  
 2   baths                                        1883 non-null   int64  
 3   sqft                                         1883 non-null   int64  
 4   year_built                                   1883 non-null   int64  
 5   restaurants                                  1883 non-null   int64  
 6   groceries                                    1883 non-null   int64  
 7   nightlife                                    1883 non-null   int64  
 8   cafes                                        1883 non-null   int64  
 9   shopping                                     1883 non-null   int64  
 10  

##Modelo

In [ ]:
#Usaremos tx_price como variable objetivo, para eso la dejamos de lado en el dataframe
y = df_filter.tx_price
X = df_filter.drop('tx_price',axis=1)

In [ ]:
X[:10]

,beds,baths,sqft,year_built,restaurants,groceries,nightlife,cafes,shopping,arts_entertainment,...,roof_Asphalt,roof_Built-up,roof_Composition,roof_Gravel/Rock,roof_Metal,roof_Other,roof_Roll Composition,roof_Shingle,roof_Slate,roof_newNAN
0,1,1,584,2013,107,9,30,19,89,6,...,0,0,0,0,0,0,0,0,0,1
1,1,1,612,1965,105,15,6,13,87,2,...,0,0,0,0,0,0,0,1,0,0
2,1,1,615,1963,183,13,31,30,101,10,...,0,0,0,0,0,0,0,0,0,1
3,1,1,618,2000,198,9,38,25,127,11,...,0,0,0,0,0,0,0,0,0,1
4,1,1,634,1992,149,7,22,20,83,10,...,0,0,0,0,0,0,0,0,0,1
5,1,1,641,1947,146,10,23,27,86,9,...,0,0,0,0,0,0,0,0,0,1
6,1,1,642,1944,159,13,36,17,92,12,...,0,0,0,0,0,0,0,0,0,1
7,1,1,650,2000,198,9,38,25,127,11,...,0,0,0,0,0,0,0,0,0,1
8,1,1,660,1983,51,8,6,2,40,18,...,0,0,0,0,0,0,0,0,0,1
9,1,1,664,1965,119,10,26,25,183,13,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
y[:10]

0    295850
1    216500
2    279900
3    379900
4    340000
5    265000
6    240000
7    388100
8    240000
9    250000
Name: tx_price, dtype: int64

In [ ]:
print(X.shape)
print(y.shape)

(1883, 42)
(1883,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1506, 42)
(377, 42)
(1506,)
(377,)


##Random Forest

In [ ]:
#Tenemos que crear una instancia de la clase random forest
rrr = RandomForestRegressor(n_estimators=100) #Los n estimators, es cuantos arboles se quiere en el bosque es decir el tamaño del modelo
rrr.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
ypred = rrr.predict(X_test)
r2_score(y_test, ypred)

#Para que sea un buen modelo tiene que dar lo más cercano a 1

0.8097848391491127

In [ ]:
#Crear una función para iterar la cantidad de estimadores
def r2_rf(num_est, Xtrain, Xtest, ytrain, ytest):
  #Crear el modelo
  model = RandomForestRegressor(n_estimators=num_est, random_state=42)

  #Ajustar el modelo al dataset (entrenar)
  model.fit(Xtrain, ytrain)

  #Testear el modelo
  #Hacer predicción con los datos de testeo
  pred = model.predict(Xtest)

  #Calcular r2 y retornar
  r2 = r2_score(ytest, pred)
  return(r2)

In [ ]:
#Verificar el funcionamiento de la función
r2_rf(100, X_train, X_test, y_train, y_test)

0.8162982019090449

In [ ]:
#Iterar sobre la cantidad de estimadores (parametros de RFR)

import time
start = time.clock()

vector_r2 = []

for est in range (50, 150):
  myr2 = r2_rf(est, X_train, X_test, y_train, y_test)
  print('Num_Est: ', est, '\t\t R2 score:', myr2)
  vector_r2.append(myr2)

print(time.clock()-start)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


Num_Est:  50 		 R2 score: 0.813366442573202
Num_Est:  51 		 R2 score: 0.8134278524218564
Num_Est:  52 		 R2 score: 0.8137543501102757
Num_Est:  53 		 R2 score: 0.8130877753330767
Num_Est:  54 		 R2 score: 0.8129923809855837
Num_Est:  55 		 R2 score: 0.8133965816580201
Num_Est:  56 		 R2 score: 0.8137250847316504
Num_Est:  57 		 R2 score: 0.8142658441141761
Num_Est:  58 		 R2 score: 0.8133971178453723
Num_Est:  59 		 R2 score: 0.8135178029749294
Num_Est:  60 		 R2 score: 0.8129811296164422
Num_Est:  61 		 R2 score: 0.8132790047381466
Num_Est:  62 		 R2 score: 0.8126552895296091
Num_Est:  63 		 R2 score: 0.8129358900701725
Num_Est:  64 		 R2 score: 0.8130716401613937
Num_Est:  65 		 R2 score: 0.8137365189488093
Num_Est:  66 		 R2 score: 0.8146481087452402
Num_Est:  67 		 R2 score: 0.8153111233566794
Num_Est:  68 		 R2 score: 0.8160630474551166
Num_Est:  69 		 R2 score: 0.8160077313227335
Num_Est:  70 		 R2 score: 0.816847094521912
Num_Est:  71 		 R2 score: 0.8162767056028161
Num_Est:  72

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


In [ ]:
best_r2_RF = max(vector_r2)
best_r2_RF

0.816847094521912

##Estandarizar Datos

In [ ]:
#Estandarizar datos
scaler = StandardScaler()
df_std = pd.DataFrame(scaler.fit_transform(df_filter), columns = df_filter.columns)

In [ ]:
df_std.head()

,tx_price,beds,baths,sqft,year_built,restaurants,groceries,nightlife,cafes,shopping,...,roof_Asphalt,roof_Built-up,roof_Composition,roof_Gravel/Rock,roof_Metal,roof_Other,roof_Roll Composition,roof_Shingle,roof_Slate,roof_newNAN
0,-0.838646,-2.265912,-1.671305,-1.305814,1.480300,1.425468,1.001140,2.945389,1.814307,0.900467,...,-0.280116,-0.032608,-0.069301,-0.12724,-0.046139,-0.163455,-0.080085,-1.435893,-0.069301,2.078271
1,-1.362677,-2.265912,-1.671305,-1.284866,-0.885332,1.382783,2.337492,0.109321,1.018267,0.863101,...,-0.280116,-0.032608,-0.069301,-0.12724,-0.046139,-0.163455,-0.080085,0.696431,-0.069301,-0.481169
2,-0.943981,-2.265912,-1.671305,-1.282621,-0.983901,3.047509,1.892041,3.063559,3.273714,1.124661,...,-0.280116,-0.032608,-0.069301,-0.12724,-0.046139,-0.163455,-0.080085,-1.435893,-0.069301,2.078271
3,-0.283576,-2.265912,-1.671305,-1.280377,0.839608,3.367648,1.001140,3.890745,2.610347,1.610414,...,-0.280116,-0.032608,-0.069301,-0.12724,-0.046139,-0.163455,-0.080085,-1.435893,-0.069301,2.078271
4,-0.547078,-2.265912,-1.671305,-1.268407,0.445336,2.321859,0.555689,2.000033,1.946980,0.788370,...,-0.280116,-0.032608,-0.069301,-0.12724,-0.046139,-0.163455,-0.080085,-1.435893,-0.069301,2.078271


In [ ]:
y_std = df_std.tx_price
X_std = df_std.drop('tx_price', axis=1)

In [ ]:
Xs_train,  Xs_test, ys_train, ys_test = train_test_split(X_std, y_std, shuffle = True , test_size=0.2, random_state=42)

##Lasso

In [ ]:
#Crear una función para iterar el parametro alpha
def r2_lasso(coef, Xtrain, Xtest, ytrain, ytest):
  #Crear el modelo
  model = Lasso(alpha = coef)

  #Ajustar el modelo al dataset (entrenar)
  model.fit(Xtrain, ytrain)

  #Testear el modelo
  #Hacer predicción con los datos de testeo
  pred = model.predict(Xtest)

  #Calcular r2 y retornar
  r2 = r2_score(ytest, pred)
  return(r2)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Iterar sobre el valor del parametro alpha

import time
start = time.clock()

vector_r2_lasso = []
alphas = [0.0005, 0.002, 0.005, 0.008, 0.08, 0.05, 0.5]

for a in alphas:
  myr2 = r2_lasso(a, X_train, X_test, y_train, y_test)
  print('Alpha: ', a, '\t\t R2 score:', myr2)
  vector_r2_lasso.append(myr2)

print(time.clock()-start)

Alpha:  0.0005 		 R2 score: 0.45639479586594145
Alpha:  0.002 		 R2 score: 0.45639478441538117
Alpha:  0.005 		 R2 score: 0.45639476151300584
Alpha:  0.008 		 R2 score: 0.4563947386089515
Alpha:  0.08 		 R2 score: 0.45639418840497836
Alpha:  0.05 		 R2 score: 0.4563944177748057
Alpha:  0.5 		 R2 score: 0.4563922366203155
1.0973880000000236


In [ ]:
best_r2_Lasso = max(vector_r2_lasso)
best_r2_Lasso

0.45639479586594145

##Ridge

In [ ]:
#Crear una función para iterar el parametro alpha
def r2_ridge(coef, Xtrain, Xtest, ytrain, ytest):
  #Crear el modelo
  model = Ridge(alpha = coef)

  #Ajustar el modelo al dataset (entrenar)
  model.fit(Xtrain, ytrain)

  #Testear el modelo
  #Hacer predicción con los datos de testeo
  pred = model.predict(Xtest)

  #Calcular r2 y retornar
  r2 = r2_score(ytest, pred)
  return(r2)

In [ ]:
#Iterar sobre el valor del parametro alpha

import time
start = time.clock()

vector_r2_ridge = []
alphas = [1, 1.5, 2, 5, 10, 30, 50, 100, 200, 500, 1000, 1500]

for a in alphas:
  myr2 = r2_ridge(a, X_train, X_test, y_train, y_test)
  print('Alpha: ', a, '\t\t R2 score:', myr2)
  vector_r2_ridge.append(myr2)

print(time.clock()-start)

Alpha:  1 		 R2 score: 0.4568658497770953
Alpha:  1.5 		 R2 score: 0.4570771959432832
Alpha:  2 		 R2 score: 0.45727039221688626
Alpha:  5 		 R2 score: 0.4581349149750382
Alpha:  10 		 R2 score: 0.45896477447247386
Alpha:  30 		 R2 score: 0.4601347012231435
Alpha:  50 		 R2 score: 0.4603571293098536
Alpha:  100 		 R2 score: 0.46002127116469604
Alpha:  200 		 R2 score: 0.4587157427959848
Alpha:  500 		 R2 score: 0.4553589178872902
Alpha:  1000 		 R2 score: 0.45199639280034454
Alpha:  1500 		 R2 score: 0.4500551880989001
0.26116300000001047


In [ ]:
best_r2_Ridge = max(vector_r2_ridge)
best_r2_Ridge

0.4603571293098536

#Gradient Boosting

In [ ]:
gbr = GradientBoostingRegressor(learning_rate=0.1, n_estimators = 100, max_depth = 3, random_state = 42)
gbr.fit(Xs_train, ys_train)

GradientBoostingRegressor(random_state=42)

In [ ]:
ypred_gbr = gbr.predict(Xs_test)
r2_score(ys_test, ypred_gbr)

0.801848009816297

In [ ]:
#Crear una función para iterar sobre los parametros
def r2_gbr(lr, est, depth, Xtrain, Xtest, ytrain, ytest):
  #Crear el modelo
  model =GradientBoostingRegressor(learning_rate=lr, n_estimators=est, max_depth=depth, random_state=42)

  #Ajustar el modelo al dataset (entrenar)
  model.fit(Xtrain, ytrain)

  #Testear el modelo
  #Hacer predicción con los datos de testeo
  pred = model.predict(Xtest)

  #Calcular r2 y retornar
  r2 = r2_score(ytest, pred)
  return(r2)

In [ ]:
import time
start = time.clock()

vector_r2_gbr = []
alphas = [0.0001,0.0005,0.008,0.01,0.1,0.3,0.5,0.8]
ests = [50,60,80,100,120,150]
depths = [1,3,5,10]

for a in alphas:
  for est in ests:
    for d in depths:
      myr2 = r2_gbr(a, est, d, X_train , X_test , y_train, y_test)
      print('Alpha: ', a, '\t\t Est: ',est,'\t\tDepth: ',d, '\t\tR2 Score: ',myr2)
      vector_r2_gbr.append(myr2)

print(time.clock()-start)

Alpha:  0.0001 		 Est:  50 		Depth:  1 		R2 Score:  -0.009943698251821953
Alpha:  0.0001 		 Est:  50 		Depth:  3 		R2 Score:  -0.006605729635344337
Alpha:  0.0001 		 Est:  50 		Depth:  5 		R2 Score:  -0.005814871560161361
Alpha:  0.0001 		 Est:  50 		Depth:  10 		R2 Score:  -0.005175454662912804
Alpha:  0.0001 		 Est:  60 		Depth:  1 		R2 Score:  -0.009209431418478742
Alpha:  0.0001 		 Est:  60 		Depth:  3 		R2 Score:  -0.005207734042792955
Alpha:  0.0001 		 Est:  60 		Depth:  5 		R2 Score:  -0.004263037923516233
Alpha:  0.0001 		 Est:  60 		Depth:  10 		R2 Score:  -0.0035395453593602966
Alpha:  0.0001 		 Est:  80 		Depth:  1 		R2 Score:  -0.007745128056070971
Alpha:  0.0001 		 Est:  80 		Depth:  3 		R2 Score:  -0.002420002287539136
Alpha:  0.0001 		 Est:  80 		Depth:  5 		R2 Score:  -0.001169334555137569
Alpha:  0.0001 		 Est:  80 		Depth:  10 		R2 Score:  -0.0002933654869254898
Alpha:  0.0001 		 Est:  100 		Depth:  1 		R2 Score:  -0.0062937652263601596
Alpha:  0.0001 		 Est:  100 		D

In [ ]:
best_r2_gbr = max(vector_r2_gbr)
best_r2_gbr

0.8162717606141281

##XGBRF Regressor

In [ ]:
xgbrf = XGBRFRegressor(learning_rate=0.1, n_estimators=100, max_depth = 3, random_state=42)
xgbrf.fit(Xs_train, ys_train)

[11:44:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRFRegressor(learning_rate=0.1, random_state=42)

In [ ]:
ypred_xgbr = xgbrf.predict(Xs_test)
r2_score(ys_test, ypred_xgbr)
#La predicción es muy mala pues es muy lejana a 1, por eso se realiza una iteración con los parámetros para saber qué parámetros son los más indicados

0.006958025035993409

In [ ]:
#Crear una función para iterar sobre los parametros
def r2_xgbr(lr, est, depth, Xtrain, Xtest, ytrain, ytest):
  #Crear el modelo
  model = XGBRFRegressor(learning_rate=lr, n_estimators=est, max_depth=depth,objective='reg:squarederror', random_state=42)#El objective se usa para evitar los warnings en la línea de código de abajo

  #Ajustar el modelo al dataset (entrenar)
  model.fit(Xtrain, ytrain)

  #Testear el modelo
  #Hacer predicción con los datos de testeo
  pred = model.predict(Xtest)

  #Calcular r2 y retornar
  r2 = r2_score(ytest, pred)
  return(r2)

In [ ]:
#Iterar sobre el valor del parámetro alpha, estimadores y depths
start = time.clock()

vector_r2_xgbr = []
alphas = [0.008,0.01,0.1,0.3,0.5,0.8] ##Nótese que se eliminaron los valores más pequeños, pues en análisis anteriores se identificó que dan valores alejados
##Por lo tanto no vale la pena usarlos
ests = [50,60,80,100,120,150]
depths = [3,5,10] ##Igualmente se elimina la profundidad 1, para ahorrar tiempo

for a in alphas:
  for est in ests:
    for d in depths:
      myr2 = r2_xgbr(a, est, d, X_train , X_test , y_train, y_test)
      print('Alpha: ', a, '\t\t Est: ',est,'\t\tDepth: ',d, '\t\tR2 Score: ',myr2)
      vector_r2_xgbr.append(myr2)

print(time.clock()-start)

Alpha:  0.008 		 Est:  50 		Depth:  3 		R2 Score:  -7.75915195893614
Alpha:  0.008 		 Est:  50 		Depth:  5 		R2 Score:  -7.758693210400237
Alpha:  0.008 		 Est:  50 		Depth:  10 		R2 Score:  -7.758720663459307
Alpha:  0.008 		 Est:  60 		Depth:  3 		R2 Score:  -7.7591611345035005
Alpha:  0.008 		 Est:  60 		Depth:  5 		R2 Score:  -7.7587188724971305
Alpha:  0.008 		 Est:  60 		Depth:  10 		R2 Score:  -7.758738516707394
Alpha:  0.008 		 Est:  80 		Depth:  3 		R2 Score:  -7.7591745846854305
Alpha:  0.008 		 Est:  80 		Depth:  5 		R2 Score:  -7.758709536548368
Alpha:  0.008 		 Est:  80 		Depth:  10 		R2 Score:  -7.7587288867280755
Alpha:  0.008 		 Est:  100 		Depth:  3 		R2 Score:  -7.759103718243974
Alpha:  0.008 		 Est:  100 		Depth:  5 		R2 Score:  -7.758619157002121
Alpha:  0.008 		 Est:  100 		Depth:  10 		R2 Score:  -7.758644993162404
Alpha:  0.008 		 Est:  120 		Depth:  3 		R2 Score:  -7.759169678013221
Alpha:  0.008 		 Est:  120 		Depth:  5 		R2 Score:  -7.758647460948529
Alpha:  

In [ ]:
best_r2_xgbr = max(vector_r2_xgbr)
best_r2_xgbr

0.32656919431126397

In [ ]:
# Saber cuál es el mejor de los valores con todos los modelos que trabajamos
models = {'Random Forest':best_r2_RF, 'Lasso':best_r2_Lasso, 'Ridge':best_r2_Ridge, 'Gradient Boosting':best_r2_gbr, 'XGRBF':best_r2_xgbr}
models

{'Random Forest': 0.816847094521912,
 'Lasso': 0.45639479586594145,
 'Ridge': 0.4603571293098536,
 'Gradient Boosting': 0.8162717606141281,
 'XGRBF': 0.32656919431126397}